<a href="https://colab.research.google.com/github/AlexTako/AI_practice/blob/main/LoRa%20%2B%20Saiga%20LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate evaluate bitsandbytes trl peft

In [ ]:
!wget https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf
!wget https://raw.githubusercontent.com/IlyaGusev/rulm/master/self_instruct/src/interact_mistral_llamacpp.py

--2024-07-13 12:14:55--  https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf
Resolving huggingface.co (huggingface.co)... 3.165.102.6, 3.165.102.58, 3.165.102.128, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.6|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/79/b3/79b3fc4694b2c3a22273003a1de570f145c14f0586c212c28c28e302adf5d3d6/2798f33ff63c791a21f05c1ee9a10bc95630b17225c140c197188a3d5cf32644?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-q4_K.gguf%3B+filename%3D%22model-q4_K.gguf%22%3B&Expires=1721132095&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMTEzMjA5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83OS9iMy83OWIzZmM0Njk0YjJjM2EyMjI3MzAwM2ExZGU1NzBmMTQ1YzE0ZjA1ODZjMjEyYzI4YzI4ZTMwMmFkZjVkM2Q2LzI3OThmMzNmZjYzYzc5MWEyMWYwNWMxZWU5YTEwYmM5NTYzMGIxNzIyNWMxNDBjMTk3MTg4YTNkNWNmMzI2NDQ%7Ec

In [ ]:
!pip install llama-cpp-python fire

In [ ]:
!python3 interact_mistral_llamacpp.py model-q4_K.gguf

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from model-q4_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llama_mode

In [ ]:
!pip install peft

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from random import randint
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline
import torch

# Идентификатор адаптера, так как мы выбрали push_to_hub=True в аргументах обучения,
# то модель по токену загрузилась на HuggingFace
peft_model_id = "IlyaGusev/saiga_mistral_7b_lora"

# Базовая модель, к которой мы обучили адаптер
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"

# # Загрузка модели с PEFT адаптером
# model = AutoPeftModelForCausalLM.from_pretrained(
#     peft_model_id,      # адаптер
#     device_map={"": 0}, # использовать GPU:0, трюк если auto приводит к ошибке
#     torch_dtype=torch.float16
# )

# # Загрузка модели с PEFT адаптером с использованием частичной загрузки
# model = AutoPeftModelForCausalLM.from_pretrained(
#     peft_model_id,
#     device_map={"": 0, "layer.0": "cpu"},  # частичная загрузка на GPU и CPU
#     torch_dtype=torch.float16
# )

# Конфигурация квантования
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

# Загрузка модели с PEFT адаптером и конфигурацией квантования
model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    quantization_config=bnb_config,
    device_map={"": 0, "layer.0": "cpu"} # частичная загрузка на GPU и CPU
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,          # базовая модель
    trust_remote_code=True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

eval_dataset = load_dataset("json", data_files="/content/ru_turbo_saiga.jsonl", split="train")
rand_idx = randint(0, len(eval_dataset))

prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=120,
               temperature=0.2, top_p=0.95, do_sample=True,
               eos_token_id=pipe.tokenizer.eos_token_id,
               pad_token_id=pipe.tokenizer.pad_token_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconFor

Generating train split: 0 examples [00:00, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [ ]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util

# Мы решили, что доверяем именно этой модели
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



In [ ]:
# Токенизируем строки
request= model.encode(eval_dataset[rand_idx]['messages'][1]['content'], convert_to_tensor=True)
answer = model.encode(eval_dataset[rand_idx]['messages'][2]['content'], convert_to_tensor=True)
predict= model.encode(outputs[0]['generated_text'][len(prompt):].strip(), convert_to_tensor=True)

# Расчитываем косинусное сходство
print(util.pytorch_cos_sim(request, answer))
print(util.pytorch_cos_sim(request, predict))
print(util.pytorch_cos_sim(answer, predict))

tensor([[0.4580]], device='cuda:0')
tensor([[0.5455]], device='cuda:0')
tensor([[0.3607]], device='cuda:0')


In [ ]:
from tqdm import tqdm  # для отрисовки прогресс бара выполнения

# Функция оценки образца
def evaluate(sample):
    # Промпт для образца
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    # Генерируем ответ
    outputs = pipe(prompt, max_new_tokens=120,
               temperature=0.2, top_p=0.95, do_sample=True,
               eos_token_id=pipe.tokenizer.eos_token_id,
               pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()

    # Токенизируем истинный и предсказанный ответы
    answer = model.encode(sample["messages"][2]["content"], convert_to_tensor=True)
    predict= model.encode(predicted_answer, convert_to_tensor=True)

    # Вычисляем косинусное сходство между истинным и предсказанным ответом
    if float(util.pytorch_cos_sim(answer, predict))>0.5:
        # Выше порога - плюс в карму точности
        return 1
    else:
        # Ниже порога - минус в карму точности
        return 0

# Массив с оценками
success_rate = []
number_of_eval_samples = 100

# Отбираем 100 записей и делаем по ним цикл с отображением прогресс бара
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    # Результаты оценки складываем в массив
    success_rate.append(evaluate(s))

# Вычисление точности
accuracy = sum(success_rate)/len(success_rate)

print(f"Точность: {accuracy*100:.2f}%")

  9%|▉         | 9/100 [02:23<22:28, 14.82s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
100%|██████████| 100/100 [24:29<00:00, 14.70s/it]

Точность: 17.00%


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_accumulated_memory_stats()
torch.cuda.reset_max_memory_cached()
del model, pipe, tokenizer